In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
import requests
import time
from bs4 import BeautifulSoup
import os

def INICIAR_DRIVER():
    options = Options()
    userAgent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15'
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument('--start-maximized')
    options.add_argument('--disable-web-security')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument('--lang=es')
    options.add_argument('--disable-blink-features=AutomationControlled')
    prefs = {'profile.default_content_setting_values.notifications': 2,'intl.accept_languages': ['es-ES', 'es']}
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    return driver


def descargar_archivo(nombre_archivo,ruta):

    cookies = {
        'JSESSIONID': 'BnEMCNSQMsgT5xwYXMl9pGIIh48xIeSzYF2gfTCx.CMA01',
        'jbosscma7101': 'ffffffffaf1398be45525d5f4f58455e445a4a422d8d',
        '__session:0.4214572102367302:selectedTab': '0',
    }
    
    headers = {
        'Accept': 'application/json',
        'Accept-Language': 'es-ES,es;q=0.9',
        'Connection': 'keep-alive',
        'Content-Type': 'application/json',
        'Referer': 'https://webcat-web.gva.es/webcat_web/datosOnlineRvvcca/cargarDatosOnlineRvvcca?languageId=es_ES)',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua': '"Not(A:Brand";v="99", "Google Chrome";v="133", "Chromium";v="133"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
    }
    
    json_data = {
        'nombreFichero': nombre_archivo,
    }
    
    response = requests.post(
        'https://webcat-web.gva.es/webcat_web/datosOnlineRvvcca/comprobarSiExisteFicheroHistorico',
        cookies=cookies,
        headers=headers,
        json=json_data,
    )
    
    cookies = {
        'JSESSIONID': 'BnEMCNSQMsgT5xwYXMl9pGIIh48xIeSzYF2gfTCx.CMA01',
        'jbosscma7101': 'ffffffffaf1398be45525d5f4f58455e445a4a422d8d',
        '__session:0.4214572102367302:selectedTab': '0',
    }
    
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'es-ES,es;q=0.9',
        'Connection': 'keep-alive',
        'Referer': 'https://webcat-web.gva.es/webcat_web/datosOnlineRvvcca/cargarDatosOnlineRvvcca?languageId=es_ES)',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15',
        'sec-ch-ua': '"Not(A:Brand";v="99", "Google Chrome";v="133", "Chromium";v="133"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
    }
    
    response = requests.get(
        'https://webcat-web.gva.es/webcat_web/datosOnlineRvvcca/descargarFicheroHistorico',
        cookies=cookies,
        headers=headers,
    )

    if response.status_code == 200:
        with open(ruta, "wb") as f:
            f.write(response.content)


driver = INICIAR_DRIVER()
driver.get('https://webcat-web.gva.es/webcat_web/datosOnlineRvvcca/cargarDatosOnlineRvvcca?languageId=es_ES)')
wait = WebDriverWait(driver, 8)

time.sleep(4)

col = driver.find_element(By.XPATH, '//*[@id="listaMunicipiosValencia"]')
zonas = col.find_elements(By.ID, 'itemListaMunicipiosValencia')

for zona in zonas:

    try:
        
        zona.click()
        time.sleep(2)
    
        dentro = driver.find_element(By.XPATH,  '//*[@id="listaEstacionesPorMunicipio"]')
        elementos = dentro.find_elements(By.ID,'idEstacionMunicipio')
    
        for elemento in elementos:
            
            detector = elemento.text
            print(detector)
            ruta1 = '/Users/pablogandia/Desktop/Analisis_aire/RawData/'+detector

            if not os.path.exists(ruta1):
                os.makedirs(ruta1)
                     
                elemento.click()
    
                input('ACEPTAR: ')
                
            
                try:
        
                    historico = driver.find_element(By.ID, 'ulTabsPestanyasEstacion')
                    option = historico.find_elements(By.CLASS_NAME, 'nav-item')[3]
                    option.click()
                    time.sleep(0.2)
    
                    anyos = driver.find_element(By.ID, 'div-tabla-historicos')
                    todos_anyos = anyos.find_elements(By.TAG_NAME, 'tr')[1:]
        
                    for anyo in todos_anyos:
                        
                        descarga_bt = anyo.find_elements(By.TAG_NAME, 'td')[2]
                        soup = BeautifulSoup(descarga_bt.get_attribute("outerHTML"), "html.parser")
                        tag = soup.find("a", class_="descargar")
                        year = tag.get("anyo_hist")
                        archivo= tag.get("nom_fich_hist")
    
                        ruta2 = ruta1+'/'+year+'.txt'
    
                        nombre_archivo = year+'/mh/'+archivo
    
                        descargar_archivo(nombre_archivo,ruta2)
                        
                except:
                    pass

            else:
                print('DONE')
            
    except:
        pass
    

In [ ]:
import csv

ruta_principal = '/Users/pablogandia/Desktop/Analisis_aire/Pablo/RawData'

for carpeta, subcarpetas, archivos in os.walk(ruta_principal):
    for archivo in archivos:
        if archivo.endswith('.txt'):
            ruta_txt = os.path.join(carpeta, archivo)
            ruta_csv = os.path.join(carpeta, archivo.replace('.txt', '.csv'))
            
            try:
                with open(ruta_txt, 'r', encoding='latin-1') as archivo_txt:
                    lineas = archivo_txt.readlines()

                for i, linea in enumerate(lineas):
                    if 'FECHA' in linea:  
                        encabezados = linea.strip().split('\t')
                        break

                datos = []

                for linea in lineas[i+1:]:
                    if linea.strip():
                        fila = linea.strip().split('\t')

                        if len(fila) == len(encabezados):
                            datos.append(fila)

                with open(ruta_csv, 'w', newline='', encoding='latin-1') as archivo_csv:
                    writer = csv.writer(archivo_csv)

                    writer.writerow(encabezados)

                    writer.writerows(datos)

                print(f"Archivo CSV creado exitosamente: {ruta_csv}")

                os.remove(ruta_txt)
                print(f"Archivo de texto eliminado: {ruta_txt}")

            except Exception as e:
                print(f"Error procesando el archivo {ruta_txt}: {e}")
